In [1]:
#Import dependencies 
import pandas as pd
import chardet
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, Numeric, ForeignKey
import os

COVID data cleaning 

In [2]:
#Import COVID CSV
with open("Resources/covid.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

data = pd.read_csv("Resources/covid.csv", encoding='UTF-8-SIG')

{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}


In [3]:
#View data table
data.head()

,date,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
0,1/21/2020,USA-53061,Snohomish,Washington,1,0.14,0.02,0,0.0,0.0
1,1/22/2020,USA-53061,Snohomish,Washington,0,0.14,0.02,0,0.0,0.0
2,1/23/2020,USA-53061,Snohomish,Washington,0,0.14,0.02,0,0.0,0.0
3,1/24/2020,USA-53061,Snohomish,Washington,0,0.14,0.02,0,0.0,0.0
4,1/24/2020,USA-17031,Cook,Illinois,1,0.14,0.00,0,0.0,0.0


In [5]:
print(data.columns.tolist())

['date', 'geoid', 'county', 'state', 'cases', 'cases_avg', 'cases_avg_per_100k', 'deaths', 'deaths_avg', 'deaths_avg_per_100k']


In [6]:
# dropping null value columns to avoid errors
data.dropna(inplace = True)

In [7]:
# new data frame with split value columns
new = data["geoid"].str.split("-", n = 1, expand = True)
  
# making separate first name column from new data frame
data["drop_USA"]= new[0]
  
# making separate last name column from new data frame
data["FIPS_Code"]= new[1]
  
# Dropping old Name columns
data.drop(columns =["drop_USA"], inplace = True)
  
# df display
data

,date,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k,FIPS_Code
0,1/21/2020,USA-53061,Snohomish,Washington,1,0.14,0.02,0,0.0,0.0,53061
1,1/22/2020,USA-53061,Snohomish,Washington,0,0.14,0.02,0,0.0,0.0,53061
2,1/23/2020,USA-53061,Snohomish,Washington,0,0.14,0.02,0,0.0,0.0,53061
3,1/24/2020,USA-53061,Snohomish,Washington,0,0.14,0.02,0,0.0,0.0,53061
4,1/24/2020,USA-17031,Cook,Illinois,1,0.14,0.00,0,0.0,0.0,17031
...,...,...,...,...,...,...,...,...,...,...,...
888550,12/31/2020,USA-69120,Tinian,Northern Mariana Islands,0,0.00,0.00,0,0.0,0.0,69120
888551,12/31/2020,USA-69110,Saipan,Northern Mariana Islands,0,0.57,1.19,0,0.0,0.0,69110
888553,12/31/2020,USA-78030,St. Thomas,Virgin Islands,8,3.71,7.19,0,0.0,0.0,78030
888554,12/31/2020,USA-78020,St. John,Virgin Islands,6,1.14,27.41,0,0.0,0.0,78020


In [8]:
print(data.columns.tolist())

['date', 'geoid', 'county', 'state', 'cases', 'cases_avg', 'cases_avg_per_100k', 'deaths', 'deaths_avg', 'deaths_avg_per_100k', 'FIPS_Code']


In [9]:
#Check data types 
data.dtypes

date                    object
geoid                   object
county                  object
state                   object
cases                    int64
cases_avg              float64
cases_avg_per_100k     float64
deaths                   int64
deaths_avg             float64
deaths_avg_per_100k    float64
FIPS_Code               object
dtype: object

In [10]:
#Convert FIPS_Code to INT
data['FIPS_Code'] = data['FIPS_Code'].astype(str).astype(int)

#Convert county and state to str
data['county'] = data['county'].astype(str)
data['state'] = data['state'].astype(str)

In [11]:
#Calculate the sum of average cases and average deaths per 100k
data = data.groupby('FIPS_Code').agg({'state' : 'first', 'county' : 'first','cases_avg_per_100k' : 'sum', 'deaths_avg_per_100k' : 'sum'})

In [12]:
data.head()

,state,county,cases_avg_per_100k,deaths_avg_per_100k
FIPS_Code,,,,
1001,Alabama,Autauga,7404.65,97.07
1003,Alabama,Baldwin,5795.35,69.76
1005,Alabama,Barbour,5827.99,129.78
1007,Alabama,Bibb,7945.89,209.43
1009,Alabama,Blount,7802.45,102.89


In [13]:
#Reset index
data = data.reset_index()
data.head()

,FIPS_Code,state,county,cases_avg_per_100k,deaths_avg_per_100k
0,1001,Alabama,Autauga,7404.65,97.07
1,1003,Alabama,Baldwin,5795.35,69.76
2,1005,Alabama,Barbour,5827.99,129.78
3,1007,Alabama,Bibb,7945.89,209.43
4,1009,Alabama,Blount,7802.45,102.89


In [14]:
#Transform COVID dataframe
data_columns = ["county", "state", "cases_avg_per_100k", "deaths_avg_per_100k","FIPS_Code"]
covid_transformed = data[data_columns].copy()
covid_transformed.head()

,county,state,cases_avg_per_100k,deaths_avg_per_100k,FIPS_Code
0,Autauga,Alabama,7404.65,97.07,1001
1,Baldwin,Alabama,5795.35,69.76,1003
2,Barbour,Alabama,5827.99,129.78,1005
3,Bibb,Alabama,7945.89,209.43,1007
4,Blount,Alabama,7802.45,102.89,1009


In [15]:
#export csv
covid_transformed.to_csv('Resources/covid_cleaned.csv', index=True)  

Education data cleaning

In [16]:
#Import file and view dataframe

file_string = "Resources/education.csv"

edf2 = pd.read_csv(file_string)
edf2

,FIPS Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,City/Suburb/Town/Rural 2013,"Less than a high school diploma, 1970","High school diploma only, 1970",...,"Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2015-19","High school diploma only, 2015-19","Some college or associate's degree, 2015-19","Bachelor's degree or higher, 2015-19","Percent of adults with less than a high school diploma, 2015-19","Percent of adults with a high school diploma only, 2015-19","Percent of adults completing some college or associate's degree, 2015-19","Percent of adults with a bachelor's degree or higher, 2015-19"
0,1007,AL,Bibb County,1.0,1.0,1.0,1.0,City,"5,272","1,402",...,20.4,7.1,"3,386","7,256","3,848","1,678",20.9,44.9,23.8,10.4
1,1009,AL,Blount County,1.0,1.0,1.0,1.0,City,"10,677","3,440",...,24.8,9.6,"7,763","13,299","13,519","5,210",19.5,33.4,34.0,13.1
2,1021,AL,Chilton County,1.0,1.0,1.0,1.0,City,"10,285","2,805",...,20.5,9.9,"5,474","13,348","7,401","3,823",18.2,44.4,24.6,12.7
3,1073,AL,Jefferson County,1.0,1.0,1.0,1.0,City,"186,882","101,656",...,28.4,24.6,"44,684","118,636","136,299","150,124",9.9,26.4,30.3,33.4
4,1115,AL,St. Clair County,1.0,1.0,1.0,1.0,City,"10,304","3,540",...,25.6,11.1,"9,488","21,849","20,207","9,880",15.4,35.6,32.9,16.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278,53000,WA,Washington,NaN,NaN,NaN,NaN,NaN,"666,656","661,800",...,34.4,27.7,"442,449","1,122,330","1,699,233","1,837,612",8.7,22.0,33.3,36.0
3279,54000,WV,West Virginia,NaN,NaN,NaN,NaN,NaN,"566,026","267,165",...,21.0,14.8,"168,624","519,091","334,314","265,398",13.1,40.3,26.0,20.6
3280,55000,WI,Wisconsin,NaN,NaN,NaN,NaN,NaN,"1,060,531","811,048",...,28.1,22.4,"308,216","1,211,981","1,244,179","1,191,329",7.8,30.6,31.5,30.1
3281,56000,WY,Wyoming,NaN,NaN,NaN,NaN,NaN,"65,252","63,630",...,35.0,21.9,"26,688","113,535","143,438","106,855",6.8,29.1,36.7,27.4


In [17]:
#Transform education data
   #Pull only desired columns
edf_filter = edf2[['FIPS Code', 'State',
                    'Percent of adults with less than a high school diploma, 2015-19', 
                   'Percent of adults with a high school diploma only, 2015-19',
                   "Percent of adults completing some college or associate's degree, 2015-19",
                  "Percent of adults with a bachelor's degree or higher, 2015-19"]].copy()
edf_filter

,FIPS Code,State,"Percent of adults with less than a high school diploma, 2015-19","Percent of adults with a high school diploma only, 2015-19","Percent of adults completing some college or associate's degree, 2015-19","Percent of adults with a bachelor's degree or higher, 2015-19"
0,1007,AL,20.9,44.9,23.8,10.4
1,1009,AL,19.5,33.4,34.0,13.1
2,1021,AL,18.2,44.4,24.6,12.7
3,1073,AL,9.9,26.4,30.3,33.4
4,1115,AL,15.4,35.6,32.9,16.1
...,...,...,...,...,...,...
3278,53000,WA,8.7,22.0,33.3,36.0
3279,54000,WV,13.1,40.3,26.0,20.6
3280,55000,WI,7.8,30.6,31.5,30.1
3281,56000,WY,6.8,29.1,36.7,27.4


Unemployment data cleaning

In [6]:
#Import file and view dataframe
unemployment_file = "Resources/unemployment2019.csv"
unemployment_df = pd.read_csv(unemployment_file)
unemployment_df.head()

,FIPS_Code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,City/Suburb/Town/Rural,Metro_2013,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,1007,AL,"Bibb County, AL",1.0,1.0,City,1.0,"8,639","8,371",268,3.1,"47,918",92.6
1,1009,AL,"Blount County, AL",1.0,1.0,City,1.0,"25,196","24,516",680,2.7,"52,902",102.2
2,1021,AL,"Chilton County, AL",1.0,1.0,City,1.0,"19,841","19,296",545,2.7,"49,692",96.0
3,1073,AL,"Jefferson County, AL",1.0,1.0,City,1.0,"316,802","307,648","9,154",2.9,"54,127",104.6
4,1115,AL,"St. Clair County, AL",1.0,1.0,City,1.0,"40,698","39,591","1,107",2.7,"65,403",126.3


In [7]:
#Drop unwanted columns
unemployment_clean_df=unemployment_df.drop(['Rural_urban_continuum_code_2013', 'Urban_influence_code_2013', 'City/Suburb/Town/Rural', 'Metro_2013', 'Civilian_labor_force_2019', 'Employed_2019', 'Median_Household_Income_2019', 'Med_HH_Income_Percent_of_State_Total_2019' ], axis=1)
unemployment_clean_df.head()

,FIPS_Code,State,Area_name,Unemployed_2019,Unemployment_rate_2019
0,1007,AL,"Bibb County, AL",268,3.1
1,1009,AL,"Blount County, AL",680,2.7
2,1021,AL,"Chilton County, AL",545,2.7
3,1073,AL,"Jefferson County, AL","9,154",2.9
4,1115,AL,"St. Clair County, AL","1,107",2.7


In [13]:
rds_connection_string = "<username>:<pw>@localhost:5432/project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
#create meta data instance
metaDataObj = MetaData()

In [119]:
#Create table in python using SQL alchemy. All tables needed FIPS_Code to be a unique primary key because of missing values
#between the tables.
education = Table(
   'education', metaDataObj, 
   Column('FIPS_Code', Integer, primary_key = True),
   Column('Less_than_HSD', Numeric), 
   Column('With_HSD', Numeric),
    Column('Some_College', Numeric),
    Column('BA_or_higher', Numeric)
)

education.create(engine, checkfirst=True)

In [120]:
#Create table in python using SQL alchemy. All tables needed FIPS_Code to be a unique primary key because of missing values
#between the tables.
covid = Table(
   'covid', metaDataObj,
    Column('county', String), 
   Column('state', String),
    Column('cases_avg_per_100k', Numeric),
    Column('deaths_avg_per_100k', Numeric),
   Column('FIPS_Code', Integer, primary_key = True)
)

covid.create(engine, checkfirst=True)

In [121]:
#Import dataFrame into sql table
covid_transformed.to_sql(name='covid', con=engine, if_exists='append', index=False)

In [122]:
#Check that data has been imported properly
pd.read_sql_query('select * from covid', con=engine).head() 

,county,state,cases_avg_per_100k,deaths_avg_per_100k,FIPS_Code
0,Autauga,Alabama,7404.65,97.07,1001
1,Baldwin,Alabama,5795.35,69.76,1003
2,Barbour,Alabama,5827.99,129.78,1005
3,Bibb,Alabama,7945.89,209.43,1007
4,Blount,Alabama,7802.45,102.89,1009


In [104]:
#last minute cleaning to match columns in Education Table
edf_filter2 = edf_filter.rename(columns={"FIPS_code": "FIPS_Code",
                   'Percent of adults with less than a high school diploma, 2015-19': 'Less_than_HSD', 
                   'Percent of adults with a high school diploma only, 2015-19': 'With_HSD',
                   "Percent of adults completing some college or associate's degree, 2015-19": 'Some_College',
                  "Percent of adults with a bachelor's degree or higher, 2015-19": "BA_or_higher" })
edf_filter2

,FIPS_Code,Less_than_HSD,With_HSD,Some_College,BA_or_higher
0,1007,20.9,44.9,23.8,10.4
1,1009,19.5,33.4,34.0,13.1
2,1021,18.2,44.4,24.6,12.7
3,1073,9.9,26.4,30.3,33.4
4,1115,15.4,35.6,32.9,16.1
...,...,...,...,...,...
3278,53000,8.7,22.0,33.3,36.0
3279,54000,13.1,40.3,26.0,20.6
3280,55000,7.8,30.6,31.5,30.1
3281,56000,6.8,29.1,36.7,27.4


In [123]:
#Import dataFrame into sql table
edf_filter2.to_sql(name='education', con=engine, if_exists='append', index=False)

In [124]:
#Check that table was imported properly
pd.read_sql_query('select * from education', con=engine).head()

,FIPS_Code,Less_than_HSD,With_HSD,Some_College,BA_or_higher
0,1007,20.9,44.9,23.8,10.4
1,1009,19.5,33.4,34.0,13.1
2,1021,18.2,44.4,24.6,12.7
3,1073,9.9,26.4,30.3,33.4
4,1115,15.4,35.6,32.9,16.1


In [129]:
#Create table in python using SQL alchemy. All tables needed FIPS_Code to be a unique primary key because of missing values
#between the tables.
unemployment = Table(
   'unemployment ', metaDataObj,
    Column('FIPS_Code', Integer, primary_key = True), 
   Column('State', String),
    Column('Area_name', String),
    Column('Unemployed_2019', Integer),
    Column('Unemployment_rate_2019', Numeric)
   
)

unemployment.create(engine, checkfirst=True)

In [14]:
#Import dataFrame into sql table
unemployment_clean_df.to_sql(name='unemployment', con=engine, if_exists='append', index=False)

In [15]:
#Check that table was imported properly
pd.read_sql_query('select * from unemployment', con=engine).head()

,FIPS_Code,State,Area_name,Unemployed_2019,Unemployment_rate_2019
0,1007,AL,"Bibb County, AL",268,3.1
1,1009,AL,"Blount County, AL",680,2.7
2,1021,AL,"Chilton County, AL",545,2.7
3,1073,AL,"Jefferson County, AL","9,154",2.9
4,1115,AL,"St. Clair County, AL","1,107",2.7
